In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import sys
import json
from time import sleep
import nltk
import numpy as np
import argparse
from langchain.llms import OpenAI
from src.baselines.baseline_utils import load_jsonl
from dotenv import load_dotenv
from types import SimpleNamespace
import asyncio
load_dotenv()

/home/d_wang/anaconda3/envs/maf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


False

In [2]:
import json

In [3]:
a = """sent2 and sent3: united states is located in the northern hemisphere and new york / new york state is a state located in the united states of america -> int1: new york is in the northern hemisphere
int1 and sent4: new york is in the northern hemisphere and June is during the summer in the northern hemisphere -> int2: June is during the summer in the northern hemisphere
sent1 and sent6 : the amount of daylight is greatest in the summer and the amount of daylight is least in the winter -> int3: there is more sunlight in the summer than in the winter
int1 and sent1: new york is in the northern hemisphere and the amount of daylight is greatest in the summer -> hypothesis: new york state has the greatest sunlight during june"""
print(json.dumps(a, indent=2))

"\nsent2 and sent3: united states is located in the northern hemisphere and new york / new york state is a state located in the united states of america -> int1: new york is in the northern hemisphere\nint1 and sent4: new york is in the northern hemisphere and June is during the summer in the northern hemisphere -> int2: June is during the summer in the northern hemisphere\nsent1 and sent6 : the amount of daylight is greatest in the summer and the amount of daylight is least in the winter -> int3: there is more sunlight in the summer than in the winter\nint1 and sent1: new york is in the northern hemisphere and the amount of daylight is greatest in the summer -> hypothesis: new york state has the greatest sunlight during june\n"


In [ ]:
args = {
    'data_dir': '../../data/gsm_data',
    'save_dir': 'models',
    'debug': False,
    'exp_label': 'default',
    'task': 'pot_gsm',
    'model': 'gpt-3.5-turbo',
    'max_tokens': 2048,
    'temperature': 0.0,
}
args['ckpt_path'] = os.path.join(args['save_dir'], args['exp_label'])
args = SimpleNamespace(**args)